Get the data from [TLC](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

In [ ]:
import pandas as pd
import requests
import io

In [ ]:
csv = pd.read_parquet('yellow_tripdata_2023-10.parquet')
csv.to_csv('yellow_tripdata_2023-10.csv')

In [ ]:
df = pd.read_csv('yellow_tripdata_2023-10.csv')

<ipython-input-3-bf2d278e10e1>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('yellow_tripdata_2023-10.csv')


In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [ ]:
df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,0,1,2023-10-01 00:16:44,2023-10-01 00:16:49,1.0,0.00,1.0,N,168,168,...,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0,0
1,1,1,2023-10-01 00:23:24,2023-10-01 00:23:47,1.0,0.00,1.0,N,168,168,...,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0,1
2,2,1,2023-10-01 00:21:18,2023-10-01 00:27:31,1.0,0.90,1.0,N,161,186,...,6.5,3.5,0.5,2.9,0.0,1.0,14.4,2.5,0.0,2
3,3,1,2023-10-01 00:17:39,2023-10-01 00:17:47,0.0,0.00,1.0,N,255,255,...,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0,3
4,4,2,2023-10-01 00:16:15,2023-10-01 00:22:57,2.0,1.41,1.0,N,151,239,...,10.0,1.0,0.5,3.0,0.0,1.0,18.0,2.5,0.0,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522285 entries, 0 to 3522284
Data columns (total 21 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   VendorID               int64  
 2   tpep_pickup_datetime   object 
 3   tpep_dropoff_datetime  object 
 4   passenger_count        float64
 5   trip_distance          float64
 6   RatecodeID             float64
 7   store_and_fwd_flag     object 
 8   PULocationID           int64  
 9   DOLocationID           int64  
 10  payment_type           int64  
 11  fare_amount            float64
 12  extra                  float64
 13  mta_tax                float64
 14  tip_amount             float64
 15  tolls_amount           float64
 16  improvement_surcharge  float64
 17  total_amount           float64
 18  congestion_surcharge   float64
 19  Airport_fee            float64
 20  trip_id                int64  
dtypes: float64(12), int64(6), object(3)
memory usage: 564.

In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [ ]:
df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,0,1,2023-10-01 00:16:44,2023-10-01 00:16:49,1.0,0.00,1.0,N,168,168,...,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0,0
1,1,1,2023-10-01 00:23:24,2023-10-01 00:23:47,1.0,0.00,1.0,N,168,168,...,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0,1
2,2,1,2023-10-01 00:21:18,2023-10-01 00:27:31,1.0,0.90,1.0,N,161,186,...,6.5,3.5,0.5,2.9,0.0,1.0,14.4,2.5,0.0,2
3,3,1,2023-10-01 00:17:39,2023-10-01 00:17:47,0.0,0.00,1.0,N,255,255,...,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0,3
4,4,2,2023-10-01 00:16:15,2023-10-01 00:22:57,2.0,1.41,1.0,N,151,239,...,10.0,1.0,0.5,3.0,0.0,1.0,18.0,2.5,0.0,4


In [ ]:
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2023-10-01 00:16:44,0,1,10,2023,6,2023-10-01 00:16:49,0,1,10,2023,6
1,1,2023-10-01 00:23:24,0,1,10,2023,6,2023-10-01 00:23:47,0,1,10,2023,6
2,2,2023-10-01 00:21:18,0,1,10,2023,6,2023-10-01 00:27:31,0,1,10,2023,6
3,3,2023-10-01 00:17:39,0,1,10,2023,6,2023-10-01 00:17:47,0,1,10,2023,6
4,4,2023-10-01 00:16:15,0,1,10,2023,6,2023-10-01 00:22:57,0,1,10,2023,6


In [ ]:
passenger_count = df[['passenger_count']].reset_index(drop=True)
passenger_count['passenger_count_id'] = passenger_count.index
passenger_count = passenger_count[['passenger_count_id','passenger_count']]

trip_distance = df[['trip_distance']].reset_index(drop=True)
trip_distance['trip_distance_id'] = trip_distance.index
trip_distance = trip_distance[['trip_distance_id','trip_distance']]

In [ ]:
passenger_count.head()

,passenger_count_id,passenger_count
0,0,1.0
1,1,1.0
2,2,1.0
3,3,0.0
4,4,2.0


In [ ]:
trip_distance.head()

,trip_distance_id,trip_distance
0,0,0.00
1,1,0.00
2,2,0.90
3,3,0.00
4,4,1.41


In [ ]:
rate_map = {
    0:"Standard rate",
    1:"JFK",
    2:"Newark",
    3:"Nassau or Westchester",
    4:"Negotiated Fare",
    5:"Group ride"
}
ratecode = df[['RatecodeID']].reset_index(drop=True)
ratecode['ratecode_id'] = ratecode.index
ratecode['ratecode_name'] = ratecode['RatecodeID'].map(rate_map)
ratecode = ratecode[['ratecode_id','RatecodeID','ratecode_name']]

ratecode.head()

,ratecode_id,RatecodeID,ratecode_name
0,0,1.0,JFK
1,1,1.0,JFK
2,2,1.0,JFK
3,3,1.0,JFK
4,4,1.0,JFK


In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment = df[['payment_type']].reset_index(drop=True)
payment['payment_id'] = payment.index
payment['payment_name'] = payment['payment_type'].map(payment_type_name)
payment = payment[['payment_id','payment_type','payment_name']]
payment.head()

,payment_id,payment_type,payment_name
0,0,2,Cash
1,1,2,Cash
2,2,1,Credit card
3,3,3,No charge
4,4,1,Credit card


In [ ]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'ratecode_id', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
               'payment_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]